In [1]:
#!/usr/bin/python

from __future__ import print_function
import time,random,threading
import tensorflow as tf
from time import sleep
import numpy as np
from vizdoom import *
import skimage.color, skimage.transform
from tqdm import tqdm
from tensorflow.python import debug as tf_debug

CONFIG_FILE_PATH = "./config/simpler_basic.cfg"
MODEL_PATH = "./model_v00/model_v00.ckpt"
RESOLUTION = (40,60,1)

N_ADV = 5

N_WORKERS = 10

WORKER_STEPS =100000

UPDATE_FREQ = 10

N_ACTION = 2
MAX_AIM = 100.0

GAMMA = 0.99

FREQ_UPDATE = 10
FREQ_TEST = 50

EPS_START = 0.5
EPS_END = 0.0
EPS_STEPS = WORKER_STEPS*N_WORKERS

LEARNING_RATE = 5e-3
RMSProbDecaly = 0.99

In [2]:
# --スレッドになるクラスです　-------
class Worker_thread:
    # スレッドは学習環境environmentを持ちます
    def __init__(self, thread_name, parameter_server):
        self.environment = Environment(thread_name, parameter_server)
        print(thread_name," Initialized")

    def run(self):            
        while True:
            if not self.environment.finished:
                self.environment.run()
            else:
                sleep(1.0)

In [3]:
class Environment(object):
    def __init__(self,name, parameter_server):
        self.game = DoomGame()
        self.game.add_available_button(Button.TURN_LEFT_RIGHT_DELTA,100)
        self.game.load_config(CONFIG_FILE_PATH)
        self.game.set_window_visible(False)
        self.game.set_mode(Mode.PLAYER)
        self.game.set_screen_format(ScreenFormat.GRAY8)
        #game.set_screen_format(ScreenFormat.CRCGCB)
        self.game.set_screen_resolution(ScreenResolution.RES_640X480)
        self.game.init()
        
        self.network = Network_local(name, parameter_server)
        self.agent = Agent(name,self.network)
        
        self.local_step = 0
        
        self.finished = False
        
        self.name = name
        
        self.record = False
        self.parameter_server = parameter_server
        
        self.loss_fire = []
        self.loss_aim = []
        self.loss_value = []
        self.frame = []
        
        self.record_reward = []
    
    def start_episode(self):
        self.game.new_episode()
        
    def preprocess(self,img):
        if len(img.shape) == 3:
            img = img.transpose(1,2,0)

        img = skimage.transform.resize(img, RESOLUTION,mode='constant')
        img = img.astype(np.float32)
        return img
    
    def run(self):
        global frames
        
        self.game.new_episode()
        
        train_episode = 0
        for step in range(WORKER_STEPS):
            
            if step % 500 == 0:
                buff = self.name + ":" + str(step) + "step is passed"
                print(buff)
            #Copy params from global
            self.agent.network.pull_parameter_server()

            if not self.game.is_episode_finished():

                s1 = self.preprocess(self.game.get_state().screen_buffer)
                action = self.agent.act(s1)
                reward = self.game.make_action(action,1)
                isterminal = self.game.is_episode_finished()
                s2 = self.preprocess(self.game.get_state().screen_buffer) if not isterminal else None
                
                if self.record==True:
                    self.parameter_server.write_summary(frames,np.array([s1]),np.array([action]),np.array([[reward]]))
                    l_a,l_f,l_v = self.parameter_server.calc_loss(frames,np.array([s1]),np.array([action]),np.array([[reward]]))
                    self.loss_fire.append(l_a[0][0])
                    self.loss_aim.append(l_f[0][0])
                    self.loss_value.append(l_v[0][0])
                    self.frame.append(frames)

                self.agent.advantage_push_network(s1,action,reward,s2,isterminal)

                frames += 1
                self.local_step += 1

            else:
                train_episode += 1
                self.start_episode()
                frames += 1
                
        print(self.name," finished |",train_episode," episodes was trained")
        
        self.finished = True
    
    def test_score(self):
        
        global frames
        current_step = frames
        self.agent.network.pull_parameter_server()

        self.game.new_episode()
        total_reward = 0
        while not self.game.is_episode_finished():

            s1 = self.preprocess(self.game.get_state().screen_buffer)                
            action = self.agent.act_test(s1)
            reward = self.game.make_action(action,1)

            total_reward += reward

        buff = "-------TEST of %s in %d step--------\n"%(self.name,current_step)
        buff += "\t REWARD: %.2f\n"%(total_reward) 
        print(buff)
        self.record_reward.append(total_reward)


In [4]:
class NetworkSetting:
    
    def state():
        name = "STATE"
        shape = [None,RESOLUTION[0],RESOLUTION[1],RESOLUTION[2]]
        return tf.placeholder(tf.float32,shape=shape,name=name)
    
    def conv1(pre_layer):
        num_outputs = 8
        kernel_size = [6,6]
        stride = [3,3]
        padding = 'SAME'
        activation = tf.nn.relu
        weights_init = tf.contrib.layers.xavier_initializer_conv2d()
        bias_init = tf.constant_initializer(0.1)
        
        return tf.contrib.layers.conv2d(pre_layer,kernel_size=kernel_size,num_outputs=num_outputs, \
                                            stride=stride,padding=padding,activation_fn=activation, \
                                           weights_initializer=weights_init, \
                                            biases_initializer=bias_init)
    
    def conv2(pre_layer):
        num_outputs = 16
        kernel_size = [3,3]
        stride = [2,2]
        padding = 'SAME'
        activation = tf.nn.relu
        weights_init = tf.contrib.layers.xavier_initializer_conv2d()
        bias_init = tf.constant_initializer(0.1)
        return tf.contrib.layers.conv2d(pre_layer,kernel_size=kernel_size,num_outputs=num_outputs, \
                                            stride=stride,padding=padding,activation_fn=activation, \
                                           weights_initializer=weights_init,biases_initializer=bias_init)
        
    def reshape(pre_layer):
        return tf.contrib.layers.flatten(pre_layer)
        
    def fc1(pre_layer):
        num_outputs = 512
        activation_fn = tf.nn.relu
        weights_init = tf.contrib.layers.xavier_initializer()
        bias_init = tf.constant_initializer(0.1)
        return tf.contrib.layers.fully_connected(pre_layer,num_outputs=num_outputs,activation_fn=activation_fn,\
                                                    weights_initializer=weights_init, biases_initializer=bias_init)
    
    def policy_mu(pre_layer):
        num_outputs = 2
        activation_fn = tf.nn.sigmoid
        weights_init = tf.contrib.layers.xavier_initializer()
        bias_init = tf.constant_initializer(0.1)
        return tf.contrib.layers.fully_connected(pre_layer,num_outputs=num_outputs,activation_fn=activation_fn,\
                                                    weights_initializer=weights_init, biases_initializer=bias_init)*200 - 100
    
    def policy_gamma(pre_layer):
        num_outputs = 2
        activation_fn = tf.nn.softplus
        weights_init = tf.contrib.layers.xavier_initializer()
        bias_init = tf.constant_initializer(0.1)
        
        return tf.sqrt(tf.contrib.layers.fully_connected(pre_layer,num_outputs=num_outputs,activation_fn=activation_fn,\
                                                    weights_initializer=weights_init, biases_initializer=bias_init))
    def value(pre_layer):
        num_outputs = 1
        activation_fn = None
        weights_init = tf.contrib.layers.xavier_initializer()
        bias_init = tf.constant_initializer(0.1)
        
        return tf.contrib.layers.fully_connected(pre_layer,num_outputs=num_outputs,activation_fn=activation_fn,\
                                                weights_initializer=weights_init, biases_initializer=bias_init)

In [5]:
# --グローバルなTensorFlowのDeep Neural Networkのクラスです　-------
class ParameterServer:
    def __init__(self):
        with tf.variable_scope("parameter_server"):      # スレッド名で重み変数に名前を与え、識別します（Name Space）
            self._build_model()            # ニューラルネットワークの形を決定
            
        with tf.variable_scope("summary"):
            self._summary()
            
        self.saver = tf.train.Saver()

        self.weights_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="parameter_server")
        self.optimizer = tf.train.RMSPropOptimizer(LEARNING_RATE, RMSProbDecaly)    # loss関数を最小化していくoptimizerの定義です
        
        print("-------GLOBAL-------")
        for w in self.weights_params:
            print(w)

    def _build_model(self):
            self.state = NetworkSetting.state()
            self.conv1 = NetworkSetting.conv1(self.state)
            self.conv2 = NetworkSetting.conv2(self.conv1)
            reshape = NetworkSetting.reshape(self.conv2)
            fc1 = NetworkSetting.fc1(reshape)

            with tf.variable_scope("policy"):
                self.mu = NetworkSetting.policy_mu(fc1)
                self.gamma = NetworkSetting.policy_gamma(fc1)
            
            with tf.variable_scope("value"):
                self.value = NetworkSetting.value(fc1)
    
    # Recording node in tensorboard
    def _summary(self):
        
        self.a_t = tf.placeholder(tf.float32, shape=(None, N_ACTION))
        self.r_t = tf.placeholder(tf.float32, shape=(None,1))

        # Normal Distributions as Policy
        p_aim = tf.distributions.Normal(loc=self.mu[:,0],scale=self.gamma[:,0])
        p_fire = tf.distributions.Normal(loc=self.mu[:,1],scale=self.gamma[:,1])

        # Probability for Action_Aim
        prob_aim = tf.reshape(p_aim.prob(self.a_t[:,0]),[-1,1],name="prob_aim")

        # Probability for Action_Fire
        cdf_fire = p_fire.cdf(0.5)
        prob_fire = cdf_fire * (tf.ones_like(self.a_t[:,1])-self.a_t[:,1]) + (tf.ones_like(self.a_t[:,1])-cdf_fire)*self.a_t[:,1]
        prob_fire = tf.reshape(prob_fire,[-1,1],name="prob_fire")

        log_policy_aim = tf.log(prob_aim + 1e-10, name="log_policy_aim")
        log_policy_fire = tf.log(prob_fire + 1e-10, name="log_policy_fire")

        advantage = self.r_t - self.value

        self.loss_policy_aim = -log_policy_aim * tf.stop_gradient(advantage)
        self.loss_policy_fire = -log_policy_fire * tf.stop_gradient(advantage)

        self.loss_value = tf.square(advantage)
        
        tf.summary.scalar('loss_aim',self.loss_policy_aim[0][0])
        tf.summary.scalar('loss_fire', self.loss_policy_fire[0][0])
        tf.summary.scalar('loss_value', self.loss_value[0][0])
        
        state_shape = self.state.get_shape()
        conv1_shape = self.conv1.get_shape()
        conv2_shape = self.conv2.get_shape()
        tf.summary.image('state',tf.reshape(self.state,[-1, state_shape[1], state_shape[2], state_shape[3]]),1)
        tf.summary.image('conv1',tf.reshape(self.conv1,[-1, conv1_shape[1], conv1_shape[2], 1]),1)
        tf.summary.image('conv2',tf.reshape(self.conv2,[-1, conv2_shape[1], conv2_shape[2], 1]),1)
        
        self.merged = tf.summary.merge_all()
        self.writer = tf.summary.FileWriter("./logs",SESS.graph)
        
    def calc_loss(self,step,s1,a,r):
        loss_aim,loss_fire,loss_v = SESS.run([self.loss_policy_aim, \
                                          self.loss_policy_fire, \
                                          self.loss_value], \
                                         feed_dict={self.state:s1,self.a_t:a,self.r_t:r})
        return loss_aim,loss_fire,loss_v
    
    def write_summary(self,step,s1,a,r):
        m = SESS.run(self.merged,feed_dict={self.state:s1,self.a_t:a,self.r_t:r})
        self.writer.add_summary(m,step)
    
    def save_model(self):
        self.saver.save(SESS, MODEL_PATH)

In [6]:
class Agent(object):
    def __init__(self,name,network):
        self.name = name
        self.network = network
        self.memory = []
    
    def act(self,s1):
        
        global frames
        
        if frames>=EPS_STEPS:
            eps = EPS_END
        else:
            eps = EPS_START + frames*(EPS_END - EPS_START) / EPS_STEPS
        
        if random.random() < eps:
            aim = random.random() * MAX_AIM
            attack = random.randint(0,1)
            return [aim,attack]
        else:
            s1 = np.array([s1])
            action_aim, action_fire = self.network.predict_actions(s1)
            return [action_aim[0],action_fire[0]]
        
    def act_test(self,s1):
        s1 = np.array([s1])
        action_aim, action_fire = self.network.predict_actions(s1)
        return [action_aim[0],action_fire[0]]
    
    def advantage_push_network(self,s1,action,reward,s2,isterminal):
        
        self.memory.append((s1,action,reward,s2))
        
        if isterminal:
            for i in range(len(self.memory)-1,-1,-1):
                s1,a,r,s2 = self.memory[i]
                if i==N_ADV-1:
                    self.R = 0
                else:
                    self.R = r + GAMMA*self.R
                
                self.network.train_push(s1,a,self.R,s2,isterminal)
            
            self.memory = []
            self.R = 0
            self.network.update_parameter_server()

        if len(self.memory)>=N_ADV:
            
            for i in range(N_ADV-1,-1,-1):
                s1,a,r,s2 = self.memory[i]
                if i==N_ADV-1:
                    self.R = self.network.predict_value(np.array([s1]))[0][0]
                else:
                    self.R = r + GAMMA*self.R
                
                self.network.train_push(s1,a,self.R,s2,isterminal)
            
            self.memory = []
            self.R = 0
            self.network.update_parameter_server()
    

In [7]:
class Network_local(object):
    def __init__(self,name,parameter_server):
        self.name = name
        with tf.variable_scope(self.name):
            self._model()
            self._build_graph(parameter_server)
            
        self.s1 = np.empty(shape=(100,RESOLUTION[0],RESOLUTION[1],RESOLUTION[2]),dtype=np.float32)
        self.s2 = np.empty(shape=(100,RESOLUTION[0],RESOLUTION[1],RESOLUTION[2]),dtype=np.float32)
        self.reward = np.empty(shape=(100,1),dtype=np.float32)
        self.action = np.empty(shape=(100,2),dtype=np.float32)
        self.isterminal = np.empty(shape=(100,1),dtype=np.int8)
        self.queue_pointer = 0
        
#         print("-----LOCAL weights---")
#         for w in self.weights_params:
#             print(w)
            
#         print("-----LOCAL grads---")
#         for w in self.grads:
#             print(w)
    
    def _model(self):
        
        self.state = NetworkSetting.state()
        conv1 = NetworkSetting.conv1(self.state)
        conv2 = NetworkSetting.conv2(conv1)
        reshape = NetworkSetting.reshape(conv2)
        fc1 = NetworkSetting.fc1(reshape)

        with tf.variable_scope("policy"):
            self.mu = NetworkSetting.policy_mu(fc1)
            self.gamma = NetworkSetting.policy_gamma(fc1)

        with tf.variable_scope("value"):
            self.value = NetworkSetting.value(fc1)
            
    def _build_graph(self,parameter_server):
#         with tf.variable_scope(self.name+"_graph"):
        self.a_t = tf.placeholder(tf.float32, shape=(None, N_ACTION))
        self.r_t = tf.placeholder(tf.float32, shape=(None,1))

        # Normal Distributions as Policy
        self.p_aim = tf.distributions.Normal(loc=self.mu[:,0],scale=self.gamma[:,0] + 0.5)
        self.p_fire = tf.distributions.Normal(loc=self.mu[:,1],scale=self.gamma[:,1] + 0.5)
        
        self.sample_aim = self.p_aim.sample([1])
        self.sample_fire = self.p_fire.sample([1])

        # Probability for Action_Aim
        prob_aim = tf.reshape(self.p_aim.prob(self.a_t[:,0]),[-1,1],name="prob_aim")

        # Probability for Action_Fire
        self.cdf_fire = self.p_fire.cdf(0.5)
        self.prob_fire = self.cdf_fire * (tf.ones_like(self.a_t[:,1])-self.a_t[:,1]) + (tf.ones_like(self.a_t[:,1])-self.cdf_fire)*self.a_t[:,1]
        self.prob_fire = tf.reshape(self.prob_fire,[-1,1],name="prob_fire")

        self.prob = tf.concat([prob_aim,self.prob_fire],1)

        log_policy_aim = tf.log(prob_aim + 1e-10, name="log_policy_aim")
        log_policy_fire = tf.log(self.prob_fire + 1e-10, name="log_policy_fire")

        advantage = self.r_t - self.value

        loss_policy_aim = -log_policy_aim * tf.stop_gradient(advantage)
        loss_policy_fire = -log_policy_fire * tf.stop_gradient(advantage)

        loss_value = tf.square(advantage)

        self.loss_total = tf.reduce_mean(loss_policy_aim + loss_policy_fire + loss_value)

        self.weights_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self.name)
        self.grads = tf.gradients(self.loss_total, self.weights_params)

        self.update_global_weight_params = \
            parameter_server.optimizer.apply_gradients(zip(self.grads, parameter_server.weights_params))

        self.pull_global_weight_params = [l_p.assign(g_p) for l_p,g_p in zip(self.weights_params,parameter_server.weights_params)]

        self.push_local_weight_params = [g_p.assign(l_p) for g_p,l_p in zip(parameter_server.weights_params,self.weights_params)]
        
    def pull_parameter_server(self):
        SESS.run(self.pull_global_weight_params)
    
    def push_parameter_server(self):
        SESS.run(self.push_local_weight_params)
        
    def show_weights(self):
        hoge = SESS.run(self.weights_params)
        for i in range(len(hoge)):
            print(hoge[i])
            
    def update_parameter_server(self):
        if self.queue_pointer > 0:
            s1 = self.s1[0:self.queue_pointer]
            s2 = self.s2[0:self.queue_pointer]
            r = self.reward[0:self.queue_pointer]
            a = self.action[0:self.queue_pointer]
            feed_dict = {self.state: s1,self.a_t:a, self.r_t:r}
            SESS.run(self.update_global_weight_params,feed_dict)
            self.queue_pointer = 0
    
    def predict_value(self,s):
        v = SESS.run(self.value,feed_dict={self.state:s})
        return v
    
    def predict_actions(self,s):
        feed_dict = {self.state:s}
        [action_aim, action_fire] = SESS.run([self.sample_aim, self.sample_fire],feed_dict)
        # Encode action_fire to 0 or 1 
        action_fire[action_fire>=0] = 1
        action_fire[action_fire<0] = 0
        return [action_aim[0],action_fire[0]]
    
    def predict_probability(self,s,a):
        feed_dict = {self.state:s, self.a_t:a}
        prob = SESS.run(self.prob, feed_dict)
        return prob
    
    def train_push(self,s,a,r,s_,isterminal):
        self.s1[self.queue_pointer] = s
        self.s2[self.queue_pointer] = s_
        self.action[self.queue_pointer] = a
        self.reward[self.queue_pointer] = r
        self.isterminal[self.queue_pointer] = isterminal
        self.queue_pointer += 1

In [8]:
# -- main ここからメイン関数です------------------------------
# global変数の定義と、セッションの開始です
frames = 0              # 全スレッドで共有して使用する総ステップ数
SESS = tf.Session()     # TensorFlowのセッション開始
isLearned = False

# M1.スレッドを作成します
with tf.device("/cpu:0"):
    parameter_server = ParameterServer()    # 全スレッドで共有するパラメータを持つエンティティです
    threads = []     # 並列して走るスレッド
    # 学習するスレッドを用意
    for i in range(N_WORKERS):
        thread_name = "local_thread"+str(i+1)
        threads.append(Worker_thread(thread_name=thread_name, parameter_server=parameter_server))
        
test_env = Environment("test_env", parameter_server)

# TensorFlowでマルチスレッドを実行します
SESS.run(tf.global_variables_initializer())     # TensorFlowを使う場合、最初に変数初期化をして、実行します

sleep(3.0)

threads[0].environment.record = True

start_time = time.time()
for worker in threads:
    job = lambda: worker.run()      # この辺は、マルチスレッドを走らせる作法だと思って良い
    t = threading.Thread(target=job)
    t.start()

test_frame = 0
while True:
    
    if frames >= test_frame and frames<test_frame+1000:
#         print(frames)
        test_env.test_score()
        test_frame += 1000
    elif frames >= test_frame+1000:
        print("TEST at %d~%d step cant be finished"%(test_frame, test_frame+1000-1))
        test_frame += 1000
    else:
        pass
    
    isLearned = True
    for worker in threads:
        if not worker.environment.finished:
            isLearned = False
    
    if isLearned:
        break

print("*****************************\nTIME to LEARNING:%.3f [sec]\n*****************************"%(time.time()-start_time))

np.save("./records/reward.npy",np.array(test_env.record_reward))
np.save("./records/loss_policy_fire.npy", threads[0].environment.loss_fire)
np.save("./records/loss_policy_aim.npy", threads[0].environment.loss_aim)
np.save("./records/loss_policy_value.npy", threads[0].environment.loss_value)
np.save("./records/frame.npy", threads[0].environment.frame)

parameter_server.save_model()
print("Learning phase is finished")
for i in range(3):
    test_env.test_score()

-------GLOBAL-------
<tf.Variable 'parameter_server/Conv/weights:0' shape=(6, 6, 1, 8) dtype=float32_ref>
<tf.Variable 'parameter_server/Conv/biases:0' shape=(8,) dtype=float32_ref>
<tf.Variable 'parameter_server/Conv_1/weights:0' shape=(3, 3, 8, 16) dtype=float32_ref>
<tf.Variable 'parameter_server/Conv_1/biases:0' shape=(16,) dtype=float32_ref>
<tf.Variable 'parameter_server/fully_connected/weights:0' shape=(1120, 512) dtype=float32_ref>
<tf.Variable 'parameter_server/fully_connected/biases:0' shape=(512,) dtype=float32_ref>
<tf.Variable 'parameter_server/policy/fully_connected/weights:0' shape=(512, 2) dtype=float32_ref>
<tf.Variable 'parameter_server/policy/fully_connected/biases:0' shape=(2,) dtype=float32_ref>
<tf.Variable 'parameter_server/policy/fully_connected_1/weights:0' shape=(512, 2) dtype=float32_ref>
<tf.Variable 'parameter_server/policy/fully_connected_1/biases:0' shape=(2,) dtype=float32_ref>
<tf.Variable 'parameter_server/value/fully_connected/weights:0' shape=(512, 1